In [1]:
import pandas as pd
import numpy as np

In [2]:
import re
dict_file = '../data/survey/7041-1996-2015_City_Survey_DataSF.xlsx'
data_dict = pd.read_excel(dict_file, 0, skiprows=3)
data_dict['Variable'] = data_dict['Variable'].str.replace('\n', '') 
data_dict = data_dict.set_index('Variable')

#keep only questions which have FDCBA grades
data_dict1 = data_dict.ix[['id', 'year', 'finweigh', 'zipcode', 'district', 'region']]
data_dict2 = data_dict[data_dict['Value Labels'].str.contains('F-Failing', flags=re.IGNORECASE).fillna(False)]
# remove column 'swcndnbd' as it's not present in data csv
data_dict2 = data_dict2.drop('swcndnbd')
data_dict = pd.concat([data_dict1, data_dict2])

In [3]:
survey_df = pd.read_csv('../data/survey/San_Francisco_City_Survey_Data_1996-2015.csv')

# keep only valid cols
survey_df = survey_df[list(set(data_dict.index) & set(survey_df.columns))]

# remove values of 6 and 7 (dont know, not provided responses)
replace_dict = {col: {6: np.nan, 7: np.nan} for col in data_dict2.index}
survey_df = survey_df.replace(replace_dict)

# consider data only from 2009 - 2015
survey_df = survey_df[survey_df['year'].isin([2009, 2011, 2013, 2015])]

# consider only 941XX zip codes for SF city
survey_df = survey_df[(survey_df['zipcode'] > 94100) & (survey_df['zipcode'] < 94200)]
print survey_df.shape

(10271, 60)


c:\users\srinivas\anaconda\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (181,183,214) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
cat_qs_counts = data_dict2['Category'].value_counts()
cat_qs_counts

Recreation and Parks     14
Infrastructure           11
City Services General    11
Libraries                 9
Transportation            8
Children and Families     1
Name: Category, dtype: int64

In [11]:
## weight for each category is the count of the number of questions
# weights = [cat_qs_counts[data_dict2.ix[col, 'Category']] for col in data_dict2.index]

# weight_mat = survey_df[data_dict2.index].notnull() * weights
# satisfaction_scores = (weight_mat * survey_df[data_dict2.index]).sum(axis=1) / weight_mat.sum(axis=1)
# scores_by_zip = satisfaction_scores.groupby(survey_df['zipcode']).mean()

In [13]:
## weight for each category is the count of the number of questions
weights = [cat_qs_counts[data_dict2.ix[col, 'Category']] for col in data_dict2.index]

# weight_mat = survey_df[data_dict2.index].notnull() / weights
satisfaction_scores = (survey_df[data_dict2.index] / weights).sum(axis=1) 
scores_by_zip = satisfaction_scores.groupby(survey_df['zipcode']).mean()

In [14]:
scores_by_zip.sort_values()

zipcode
94105.0    7.361457
94158.0    7.497538
94109.0    7.644697
94104.0    7.649829
94123.0    7.880494
94102.0    7.891319
94120.0    7.990204
94107.0    7.992125
94103.0    8.022743
94133.0    8.074360
94131.0    8.090366
94114.0    8.115738
94115.0    8.133479
94118.0    8.205283
94101.0    8.337101
94117.0    8.346542
94111.0    8.370478
94124.0    8.370736
94122.0    8.385986
94106.0    8.399158
94121.0    8.404675
94110.0    8.413215
94108.0    8.492486
94125.0    8.496970
94132.0    8.520045
94127.0    8.544668
94126.0    8.636632
94113.0    8.642738
94130.0    8.693164
94112.0    8.710941
94134.0    8.754626
94116.0    8.828484
94119.0    8.899250
94129.0    9.068667
dtype: float64